In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, Embedding
from keras.optimizers import RMSprop, Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import json

/home/paperspace/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load data
with open("clean.json") as f:
    clean = json.load(f)
with open("buggy.json") as f:
    buggy = json.load(f)
with open("py2vec_modelJ.json") as f:
    embs = json.load(f)

In [3]:
# Lưu trữ thành mảng, lúc này các đoạn mã vẫn là các chuỗi.
clean = np.asarray(clean)
buggy = np.asarray(buggy)
# Tạo nhãn cho mỗi tiêu đề, 1 cho mã lỗi và 0 cho mã sạch
buggy_labels = np.ones(len(buggy))
clean_labels = np.zeros(len(clean))

In [4]:
# Phần nhúng hiện được lưu trữ trong từ điển và cần biến nó thành một mảng 2 chiều có kích thước (số nút, độ dài của Embedding)
# word_to_int sẽ lấy một nút và chuyển từ đó thành chỉ mục của từ đó thành ma trận nhúng
# int_to_word được làm ngược lại - lấy một chỉ mục và chuyển đổi nó trở lại một nút
embedding_matrix = []
int_to_word = []
word_to_int = {}
i = 0
for word, emb in embs.items():
    embedding_matrix.append(emb)
    int_to_word.append(word)
    word_to_int[word] = i
    i += 1
    
embedding_matrix.append(np.zeros(100)) # For unknown words we use an array of zeros.
embedding_matrix = np.asarray(embedding_matrix)
print(word_to_int['safemax'])
print(int_to_word[2])
print(np.array_equal(embs['safemax'], embedding_matrix[2]))

2
safemax
True


In [5]:
# sắp xếp lại data và nhãn
train_data = np.concatenate((clean, buggy), axis=0)
train_labels = np.concatenate((clean_labels, buggy_labels), axis=0)

# Sắp xếp lại data
for i in range(train_data.shape[0]):
    string = ''
    for j in range(len(train_data[i])):
        string += train_data[i][j] + ' '
    train_data[i] = string

# Sử dụng hàm random để xáo trộn (shuffle) dữ liệu trong train_data và train_labels một cách ngẫu nhiên, 
# nhưng đảm bảo rằng sự tương ứng giữa các phần tử trong hai mảng này được duy trì.
np.random.seed(3)
np.random.shuffle(train_data)
np.random.seed(3)
np.random.shuffle(train_labels)

In [6]:
# Lấy thử nghiệm 1000 mẫu lỗi 
test_data = train_data[train_data.shape[0]-1000:]
test_labels = train_labels[train_labels.shape[0]-1000:]
train_data = train_data[:train_data.shape[0]-1000]
train_labels = train_labels[:train_labels.shape[0]-1000]

num_words = len(embs)

In [7]:
# Chuyển đổi từng tiêu đề của chuỗi thành số nguyên - mỗi từ được biến thành chỉ mục của nó thành ma trận nhúng.
train_data_tokens = []
test_data_tokens = []
num_words_missed = 0
num_words_found = 0
for i in range(train_data.shape[0]):
    train_data_tokens.append([])
    for word in train_data[i].split():
        if word.lower() in embs:
            train_data_tokens[i].append(word_to_int[word.lower()])
            num_words_found += 1
        else:
            train_data_tokens[i].append(-1)
            num_words_missed += 1
for i in range(test_data.shape[0]):
    test_data_tokens.append([])
    for word in test_data[i].split():
        if word.lower() in embs:
            test_data_tokens[i].append(word_to_int[word.lower()])
            num_words_found += 1
        else:
            test_data_tokens[i].append(embedding_matrix.shape[0]-1)
            num_words_missed += 1
print("Tổng số lượng nút %d" % num_words_found)
print("Số nút lỗi %d" % num_words_missed)

Tổng số lượng nút 497424
Số nút lỗi 13988


In [ ]:
print(train_data_tokens[0])
print(test_data_tokens[0])
print(embedding_matrix.shape[0])

In [10]:
# Chuyển đổi một chuỗi các chỉ mục (tokens) trở lại thành một chuỗi văn bản ban đầu
# Mục đích là để hiển thị lại tiêu đề của một văn bản trong tập huấn luyện dưới dạng chuỗi văn bản, 
# từ đó có thể đọc và kiểm tra kết quả của quá trình xử lý và huấn luyện mô hình
print(train_data_tokens[2])
int_to_word.append("unknown")
def tokens_to_string(tokens):
    words = [int_to_word[token] for token in tokens if token != 0]
    text = " ".join(words)
    return text
print(tokens_to_string(train_data_tokens[2]))

[2526, 2529, 4068, 4068, 5340, 994, 3984, 2561, 2905, 4603, 5918, 5626, 5934]
_atsignsymbol_ gwtincompatible _divide_ _divide_ doublemath _dispatch_ roundtoint _openparen_ double _comma_ roundingmode _closeparen_ unknown


In [11]:
# Tính tổng số token
num_tokens = [len(tokens) for tokens in train_data_tokens + test_data_tokens]
num_tokens = np.asarray(num_tokens)

In [12]:
np.mean(num_tokens)

14.09547433989306

In [ ]:
np.max(num_tokens)

In [14]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

27

In [15]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9382613968358966

In [16]:
max_tokens = np.max(num_tokens)

In [17]:
pad = 'pre'

In [18]:
# Tensorflow yêu cầu mỗi title đều phải bằng nhau, tôi thực hiện việc căn chỉnh độ dài của các title thành một độ dài cố định
train_data_pad = pad_sequences(train_data_tokens, maxlen=max_tokens,
                              padding=pad, truncating=pad)
test_data_pad = pad_sequences(test_data_tokens, maxlen=max_tokens,
                             padding=pad, truncating=pad)

In [19]:
train_data_pad.shape

(35282, 54)

In [20]:
np.array(train_data_pad[0])

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,  400, 5404,
       3580, 2298, 2561, 3877, 4603, 1649, 5626, 4029,  994, 5934],
      dtype=int32)

In [21]:
# Create Network
from keras.layers import Dropout
num_words = len(int_to_word)
model = Sequential()
model.add(Embedding(input_dim=embedding_matrix.shape[0],
                   output_dim=embedding_matrix.shape[1],
                   input_length=max_tokens,
                   weights=[embedding_matrix],
                   trainable=False,
                   name='embedding_layer'))
model.add(CuDNNLSTM(16, return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(8))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
             optimizer=optimizer,
             metrics=['accuracy'])

Instructions for updating:
Use the retry module or similar alternatives.


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 54, 100)           593500    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 54, 16)            7552      
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 16)            0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 8)                 832       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 601,893
Trainable params: 8,393
Non-trainable params: 593,500
_________________________________________________________________


In [23]:
# Train
%%time
model.fit(train_data_pad, train_labels,
         validation_split=0.05, epochs=10, batch_size=64)

Train on 33517 samples, validate on 1765 samples
Epoch 1/10
33517/33517 [==============================] - 8s 237us/step - loss: 0.6935 - acc: 0.5052 - val_loss: 0.6898 - val_acc: 0.5439
Epoch 2/10
33517/33517 [==============================] - 6s 168us/step - loss: 0.6155 - acc: 0.6462 - val_loss: 0.5089 - val_acc: 0.7275
Epoch 3/10
33517/33517 [==============================] - 6s 172us/step - loss: 0.4859 - acc: 0.7485 - val_loss: 0.4309 - val_acc: 0.7841
Epoch 4/10
33517/33517 [==============================] - 6s 170us/step - loss: 0.4358 - acc: 0.7776 - val_loss: 0.3932 - val_acc: 0.7977
Epoch 5/10
33517/33517 [==============================] - 6s 168us/step - loss: 0.4088 - acc: 0.7922 - val_loss: 0.3757 - val_acc: 0.8062
Epoch 6/10
33517/33517 [==============================] - 6s 166us/step - loss: 0.3913 - acc: 0.7984 - val_loss: 0.3649 - val_acc: 0.8125
Epoch 7/10
33517/33517 [==============================] - 6s 168us/step - loss: 0.3741 - acc: 0.8056 - val_loss: 0.3518 - v

In [24]:
# Test trên testing data
result = model.evaluate(test_data_pad, test_labels)

1000/1000 [==============================] - 0s 152us/step


In [25]:
print("accuracy: {0:.2%}".format(result[1]))

accuracy: 80.10%
